In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import healpy as hp
import numpy as np
import pylab as pl
import camb 
import glob
rot = hp.Rotator(coord=['C','G'])
from astropy.coordinates import SkyCoord
from astropy.io import fits
import pymaster as nmt
import sys
from scipy import stats
import copy
import pickle as pkl
sys.path.append('../code')
#sys.path.append('/Users/gfabbian/Software/PolSpice_v03-07-05/bin')
from lensing_qso_cross_utils import *
import scipy.stats as sts

rot = hp.Rotator(coord=['C','G'])

In [3]:
nside=256
lmax=3*nside-1

In [4]:
def prepare_gaia_catalog(gaia_fname,verbose=False,snr_cut=0.):# Prepare raw GAIA data
    if verbose:
        print("Read catalog and convert coordinate")
    d=fits.open(gaia_fname)

    z = d[1].data['redshift_spz']
    try:
        zerr = d[1].data['redshift_spz_err']
        snr = z/zerr
    except:
        # works for old routines
        zerr=np.zeros(len(z))
        snr = 3*np.ones(len(z))
    sc = SkyCoord(ra=d[1].data["ra"], dec=d[1].data["dec"], unit='deg', frame='icrs', equinox='J2000.0')
    gs = sc.transform_to(frame='galactic')
    l = gs.l.value[snr>snr_cut] 
    b = gs.b.value[snr>snr_cut] 
    nqso = len(l)
    return z[snr>snr_cut] ,zerr[snr>snr_cut] ,l,b,nqso

In [5]:
# old catalogs
z,zerr,l,b,nqso = prepare_gaia_catalog('../../catalog_spz/gaia_spz_G20.0.fits',snr_cut=2.)

# new catalogs
z20,zerr20,l20,b20,nqso20 = prepare_gaia_catalog('../../catalog_20230406/catalog_G20.0.fits',snr_cut=2.)
z205,zerr205,l205,b205,nqso205 = prepare_gaia_catalog('../../catalog_20230406/catalog_G20.5.fits',snr_cut=2.)

In [22]:
def counts_selcorrected(l,b,selection,weight=None,verbose=False):
    selection_mask = selection>0
    m=make_counts(nside,l,b,weight=weight)
    msel=np.zeros(hp.nside2npix(nside))
    msel[selection_mask] = m[selection_mask]/selection[selection_mask]
    return msel

def process_catalog(l,b,selection,nbar_confidence_mask=None,weight=None,verbose=True):
    msel=counts_selcorrected(l,b,selection,weight)
    selection_mask = selection>0
    if nbar_confidence_mask is None:
        nbar = np.mean(msel[selection_mask]) 
    else:
        nbar = np.mean(msel[selection_mask&nbar_confidence_mask]) 
    if verbose:
        print("nbar",nbar)
    csel = overdensity_from_counts(msel,nbar,verbose=False) 
    return csel


def process_catalog_splits(l,b,selection,nbar_confidence_mask=None,nreal=1,weight=None):
    selection_mask = selection>0
    if nbar_confidence_mask is not None:
        selection_mask=selection_mask&nbar_confidence_mask
    data=[]
    for i in range(nreal):
        split_reshuffle = np.arange(len(l))
        np.random.shuffle(split_reshuffle)    

        # splits
        m1sel = counts_selcorrected((l[split_reshuffle])[0::2],(b[split_reshuffle])[0::2],selection,weight=weight)
        m2sel = counts_selcorrected((l[split_reshuffle])[1::2],(b[split_reshuffle])[1::2],selection,weight=weight)
    
        c1sel = overdensity_from_counts(m1sel,selection_mask,verbose=False) 
        c2sel = overdensity_from_counts(m2sel,selection_mask,verbose=False) 
        jksel =(c2sel-c1sel)/2
        
        data.append([c1sel,c2sel,jksel])
    if nreal==1:
        return data[0]
    else:
        return data

def process_catalog_and_splits(l,b,selection,nbar_confidence_mask=None,weight=None,verbose=False):
    c =process_catalog(l,b,selection,nbar_confidence_mask=nbar_confidence_mask,weight=weight,verbose=verbose)
    c1,c2,cjk =process_catalog_splits(l,b,selection,nbar_confidence_mask=nbar_confidence_mask,nreal=1,weight=weight)    
    return c,c1,c2,cjk

In [23]:
# read data 

# compute galactic masks
galmask80_lr= hp.read_map('../../planck_galmask80.fits')
galmask90_lr= hp.read_map('../../planck_galmask90.fits')
galmask70_lr= hp.read_map('../../planck_galmask70.fits')
galmask60_lr  = hp.read_map('../../planck_galmask60.fits')
galmask50_lr= hp.read_map('../../planck_galmask50.fits')
galmask40_lr= hp.read_map('../../planck_galmask40.fits')
galmask20_lr= hp.read_map('../../planck_galmask20.fits')

# read lensing mask: NB consisent with galmask70 + point source masks
lensmask_lr = hp.read_map('../../planck_lensmask.fits')


# Magellanic Cloud mask
mclouds = [(280.4652,-32.8884),(302.8084,-44.3277)] #Large MC and Small MC coordinates
r_mclouds = [4,2] #deg
#r_mclouds = [5,2] #deg
mclouds_mask = np.ones(hp.nside2npix(nside))
for i,lmc in enumerate(mclouds):
    mcpix = hp.query_disc(nside,hp.ang2vec(lmc[0],lmc[1],lonlat=True),np.deg2rad(r_mclouds[i]))
    mclouds_mask[mcpix]=0.

In [24]:
#NOTE # binmask is consistent with galmask80_lr of Planck


# selection function and binary mask
mask_c = hp.ud_grade(hp.read_map('../../catalog_spz/map_probability_dust_stars_m10_NSIDE64_G20.0.fits'),nside_out=nside)
sel = rot.rotate_map_pixel(mask_c)
binmask = sel>0
binmask = (sel>0)*galmask80_lr

# selection function and binary mask
mask_c204 = hp.ud_grade(hp.read_map('../../catalog_spz/map_probability_dust_stars_m10_NSIDE64_G20.4.fits'),nside_out=nside)
sel204 = rot.rotate_map_pixel(mask_c204)
binmask204 = sel204>0
binmask204 = (sel204>0)*galmask80_lr

# selection function and binary mask new catalog
mask_c20 = hp.ud_grade(hp.read_map('../../catalog_20230406/selection_function_NSIDE64_G20.0.fits'),nside_out=nside)
sel20 = rot.rotate_map_pixel(mask_c20)
binmask20 = sel20>0
binmask20 = (sel20>0)*galmask80_lr


# selection function and binary mask new catalog
mask_c205 = hp.ud_grade(hp.read_map('../../catalog_20230406/selection_function_NSIDE64_G20.5.fits'),nside_out=nside)
sel205 = rot.rotate_map_pixel(mask_c205)
binmask205 = sel205>0
binmask205 = (sel205>0)*galmask80_lr


# Make maps

In [137]:
# mask regions with low selection function 
sel_threshold = 0.5

mygalmask = (sel>sel_threshold) & galmask60_lr.astype(bool)
mygalmask20 = (sel20>sel_threshold) & galmask60_lr.astype(bool)
mygalmask205 = (sel205>sel_threshold) & galmask60_lr.astype(bool)


# old catalog
c,c1,c2,cjk   = process_catalog_and_splits(l,b,sel,mygalmask.astype(bool)&mclouds_mask.astype(bool))

c_nosel,c1_nosel,c2_nosel,cjk_nosel   = process_catalog_and_splits(l,b,np.ones(hp.nside2npix(nside)),
                                                    mygalmask.astype(bool)&mclouds_mask.astype(bool))


# new catalog 
c20,c1_20,c2_20,cjk_20   = process_catalog_and_splits(l20,b20,sel20,mygalmask20.astype(bool)&mclouds_mask.astype(bool))
c20_nosel,c1_20_nosel,c2_20_nosel,cjk_20_nosel   = process_catalog_and_splits(l20,b20,np.ones(hp.nside2npix(nside)),
                                                    mygalmask20.astype(bool)&mclouds_mask.astype(bool))

# splits catalog in redsfhit bins
c20ulr,c1ulr_20,c2ulr_20,cjkulr_20   = process_catalog_and_splits(l20[z20<0.5],b20[z20<0.5],sel20,mygalmask20.astype(bool)&mclouds_mask.astype(bool))
c20lr,c1lr_20,c2lr_20,cjklr_20   = process_catalog_and_splits(l20[z20<1],b20[z20<1],sel20,mygalmask20.astype(bool)&mclouds_mask.astype(bool))
c20hr,c1hr_20,c2hr_20,cjkhr_20   = process_catalog_and_splits(l20[z20>2],b20[z20>2],sel20,mygalmask20.astype(bool)&mclouds_mask.astype(bool))
c20uhr,c1uhr_20,c2uhr_20,cjkuhr_20   = process_catalog_and_splits(l20[z20>3],b20[z20>3],sel20,mygalmask20.astype(bool)&mclouds_mask.astype(bool))


# new catalog G20.5
c_205,c1_205,c2_205,cjk_205 = process_catalog_and_splits(l205,b205,sel205,mygalmask205.astype(bool)&mclouds_mask.astype(bool))
c_205_nosel,c1_205_nosel,c2_205,cjk_205 = process_catalog_and_splits(l205,b205,np.ones(hp.nside2npix(nside)),
                                                    mygalmask20.astype(bool)&mclouds_mask.astype(bool))

# Compute dipole

In [139]:

def process_dipole(dipole,norm =1,verbose=True):
    #mono,dipole = hp.fit_dipole(masked_map)#,gal_cut)
    lon, lat = hp.rotator.vec2dir(dipole, lonlat=True)
    amp =  np.sqrt((dipole * dipole).sum())
    lon = (360+lon)%360
    if verbose:
        print('Dipole l %.3f b %.3f A %.5e'%(lon,lat,amp/norm))
    return [lon,lat,amp/norm]

def compute_dipole(cmap,cmask,verbose=True):
    c_nodip = np.array(cmap)
    c_nodip[cmask==0]=hp.UNSEEN
    c_nodip,mono,dip=hp.remove_dipole(c_nodip,fitval=True)
    dip = process_dipole(dip,norm=1,verbose=verbose)    
    return c_nodip,mono,dip

print("G20 old / G20 / G20.5")
dipole_c = compute_dipole(c,mygalmask&mclouds_mask.astype(bool))
dipole_c20 = compute_dipole(c20,mygalmask20&mclouds_mask.astype(bool))
#dipole_c20_iter2 = compute_dipole(dipole_c20[0],mygalmask20&mclouds_mask.astype(bool))
dipole_c205 = compute_dipole(c_205,mygalmask205&mclouds_mask.astype(bool))

print("\nNo sel correction")
dipole_c_nosel = compute_dipole(c_nosel,mygalmask&mclouds_mask.astype(bool))
dipole_c20_nosel = compute_dipole(c20_nosel,mygalmask20&mclouds_mask.astype(bool))
dipole_c205_nosel = compute_dipole(c_205_nosel,mygalmask205&mclouds_mask.astype(bool))

print("\njk map")
dipole_cjk = compute_dipole(cjk,mygalmask&mclouds_mask.astype(bool))
dipole_c20jk = compute_dipole(cjk_20,mygalmask20&mclouds_mask.astype(bool))
dipole_c205jk = compute_dipole(cjk_205,mygalmask205&mclouds_mask.astype(bool))


print("\nG20 redshift bins z<0.5 / z<1 / z>2 / z>3")
dipole_c20ulr = compute_dipole(c20ulr,mygalmask20&mclouds_mask.astype(bool))
dipole_c20lr = compute_dipole(c20lr,mygalmask20&mclouds_mask.astype(bool))
dipole_c20hr = compute_dipole(c20hr,mygalmask20&mclouds_mask.astype(bool))
dipole_c20uhr = compute_dipole(c20uhr,mygalmask20&mclouds_mask.astype(bool))

G20 old / G20 / G20.5
Dipole l 314.414 b 25.197 A 3.10097e-02
Dipole l 324.453 b 27.501 A 2.01355e-02
Dipole l 349.785 b 24.967 A 2.71874e-02

No sel correction
Dipole l 201.102 b 19.857 A 6.49143e-02
Dipole l 205.858 b 24.810 A 4.37435e-02
Dipole l 196.189 b 29.384 A 4.03015e-02

jk map
Dipole l 81.619 b 1.107 A 8.28560e-03
Dipole l 351.507 b -38.876 A 4.11322e-03
Dipole l 136.492 b 31.068 A 5.30054e-03

G20 redshift bins z<0.5 / z<1 / z>2 / z>3
Dipole l 22.576 b 24.465 A 3.43893e-02
Dipole l 338.358 b 20.765 A 2.83989e-02
Dipole l 263.497 b 35.979 A 1.93645e-02
Dipole l 54.988 b 65.714 A 3.14317e-02


In [145]:

#vec_gaia = hp.ang2vec(310.127,23.368,lonlat=True)

vec_wise21 = hp.ang2vec(238.2, 28.8,lonlat=True) # dipole WISE 
vec_wise23= hp.ang2vec(237.2, 41.8,lonlat=True) # dipole WISE 
vec_cmb  = hp.ang2vec(264.021, 48.25,lonlat=True) # dipole CMB
print("angle btw wise qso and cmb dipole 2021   ",np.rad2deg(np.arccos(np.dot(vec_wise21,vec_cmb))))
print("angle btw wise qso and cmb dipole 2023   ",np.rad2deg(np.arccos(np.dot(vec_wise23,vec_cmb))))
print("angle btw wise qso 2021 and wise qso 2024",np.rad2deg(np.arccos(np.dot(vec_wise21,vec_wise23))))
print()
print("G20 old / G20 / G20.5")
for dipole in [dipole_c,dipole_c20,dipole_c205]:
    vec_gaia = hp.ang2vec(dipole[-1][0],dipole[-1][1],lonlat=True)
    print("angle btw gaia and wise qso dipole 2021",np.rad2deg(np.arccos(np.dot(vec_gaia,vec_wise21))))
    print("angle btw gaia and wise qso dipole 2023",np.rad2deg(np.arccos(np.dot(vec_gaia,vec_wise23))))    
    print("angle btw gaia and cmb dipole",np.rad2deg(np.arccos(np.dot(vec_gaia,vec_cmb))))
    
    print()

print("no sel G20 old / G20 / G20.5")
for dipole in [dipole_c_nosel,dipole_c20_nosel,dipole_c205_nosel]:
    vec_gaia = hp.ang2vec(dipole[-1][0],dipole[-1][1],lonlat=True)
    print("angle btw gaia and wise qso dipole 2021",np.rad2deg(np.arccos(np.dot(vec_gaia,vec_wise21))))
    print("angle btw gaia and wise qso dipole 2023",np.rad2deg(np.arccos(np.dot(vec_gaia,vec_wise23))))        
    print("angle btw gaia and cmb dipole",np.rad2deg(np.arccos(np.dot(vec_gaia,vec_cmb))))
    print()

print("z trend G20 redshift bins z<0.5 / z<1 / z>2 / z>3")
for dipole in [dipole_c20ulr,dipole_c20lr,dipole_c20hr,dipole_c20uhr]:
    vec_gaia = hp.ang2vec(dipole[-1][0],dipole[-1][1],lonlat=True)
    print("angle btw gaia and wise qso dipole 2021",np.rad2deg(np.arccos(np.dot(vec_gaia,vec_wise21))))
    print("angle btw gaia and wise qso dipole 2023",np.rad2deg(np.arccos(np.dot(vec_gaia,vec_wise23))))        
    print("angle btw gaia and cmb dipole",np.rad2deg(np.arccos(np.dot(vec_gaia,vec_cmb))))
    print()


angle btw wise qso and cmb dipole 2021    27.78861043532585
angle btw wise qso and cmb dipole 2023    19.903399024456245
angle btw wise qso 2021 and wise qso 2024 13.025317609763519

G20 old / G20 / G20.5
angle btw gaia and wise qso dipole 2021 66.79310910070542
angle btw gaia and wise qso dipole 2023 64.33881208918262
angle btw gaia and cmb dipole 45.433280402849775

angle btw gaia and wise qso dipole 2021 74.14175441944596
angle btw gaia and wise qso dipole 2023 70.15494507868208
angle btw gaia and cmb dipole 50.508594583210765

angle btw gaia and wise qso dipole 2021 95.10090862408087
angle btw gaia and wise qso dipole 2023 88.7515580030799
angle btw gaia and cmb dipole 68.9309937827911

no sel G20 old / G20 / G20.5
angle btw gaia and wise qso dipole 2021 34.812271798554804
angle btw gaia and wise qso dipole 2023 37.538891954975504
angle btw gaia and cmb dipole 57.416262372497

angle btw gaia and wise qso dipole 2021 29.051641077816367
angle btw gaia and wise qso dipole 2023 30.9500

# Summary

The $z>2$ sample of the quasar seems to be close to the closest to the CMB direction. To be investigated as a function of sky cuts etc as in Dam+ https://arxiv.org/pdf/2212.07733.pdf . for the $z>2$ the amplitude seems to agree with Dam+ that found $19\times 10^{-3}$.



# TODO

Perhaps we can look at the cross-matched sample and estimate $x$ and $\alpha$ exponents to have a sense if the value makes sense.